In [1]:
import torch
import deepxde as dde
import numpy as np
from typing import cast, Any
import plotly.graph_objects as go
import plotly

%load_ext jupyter_black

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [5]:
class LaminarFlow2D:
    """Class modeling the laminar flow in 2+1 dimensions.


    Args:
        rho (float): Fluid density.
        u_inf (float): Velocity of the fluid at the inlet.
        reynolds (float): Reynolds number.
    """

    __slots__ = ("rho", "u_inf", "reynolds", "nu")

    def __init__(self, rho: float, u_inf: float, reynolds: float):
        # PDE parameters
        self.rho = rho
        self.u_inf = u_inf
        self.reynolds = reynolds
        self.nu = self.u_inf / self.reynolds

    def equation(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        """Defines the ODE.

        Args:
            x (torch.Tensor): Input of the neural network.
            y (torch.Tensor): Output of the neural network.

        Returns:
            (torch.Tensor): Residual of the differential equation.

        """
        ux, uy, p = y[:, 0:1], y[:, 1:2], y[:, 2:]

        ux_x = cast(torch.Tensor, dde.grad.jacobian(y, x, i=0, j=0))
        ux_y = cast(torch.Tensor, dde.grad.jacobian(y, x, i=0, j=1))
        ux_t = cast(torch.Tensor, dde.grad.jacobian(y, x, i=0, j=2))
        uy_x = cast(torch.Tensor, dde.grad.jacobian(y, x, i=1, j=0))
        uy_y = cast(torch.Tensor, dde.grad.jacobian(y, x, i=1, j=1))
        uy_t = cast(torch.Tensor, dde.grad.jacobian(y, x, i=1, j=2))
        p_x = cast(torch.Tensor, dde.grad.jacobian(y, x, i=2, j=0))
        p_y = cast(torch.Tensor, dde.grad.jacobian(y, x, i=2, j=1))

        ux_xx = cast(torch.Tensor, dde.grad.hessian(y, x, i=0, j=0, component=0))
        ux_xy = cast(torch.Tensor, dde.grad.hessian(y, x, i=0, j=1, component=0))
        ux_yy = cast(torch.Tensor, dde.grad.hessian(y, x, i=1, j=1, component=0))
        uy_xx = cast(torch.Tensor, dde.grad.hessian(y, x, i=0, j=0, component=1))
        uy_xy = cast(torch.Tensor, dde.grad.hessian(y, x, i=0, j=1, component=1))
        uy_yy = cast(torch.Tensor, dde.grad.hessian(y, x, i=1, j=1, component=1))

        pde = [
            ux_t + ux * ux_x + uy * ux_y + p_x / self.rho - self.nu * (ux_xx + ux_yy),
            uy_t + ux * uy_x + uy * uy_y + p_y / self.rho - self.nu * (uy_xx + uy_yy),
            ux_x + uy_y,
        ]

        return pde

In [90]:
# Equation parameters

rho = 1
u_inf = 1
reynolds = 0.1
radius = 0.1

# Neural network parameters
input_size = [3]
output_size = [3]
layers_sizes = input_size + [16, 32, 16] + output_size
activation = "tanh"
initializer = "Glorot normal"

# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    # loss_weights=[0.001, 1, 1],
)
n_iterations = 10_000

# Mesh parameters
n_training_inside = 5000
n_training_bdy = 500
n_training_initial = 500
# n_test = 5000

# Ranges
x_begin = 0
x_end = 2
y_begin = 0
y_end = 1
t_begin = 0
t_end = 1


def solve_problem(
    reynolds: float,
) -> tuple[LaminarFlow2D, dde.Model, dde.model.LossHistory, dde.model.TrainState]:
    """Given the parameters, defines the model and trains the model."""
    # System
    laminar_eq = LaminarFlow2D(rho=rho, u_inf=u_inf, reynolds=reynolds)

    # Geometry description
    outer = dde.geometry.Rectangle(xmin=[x_begin, y_begin], xmax=[x_end, y_end])
    inner = dde.geometry.Disk([0.5, 0.5], radius)
    geom_space = outer
    # geom_space = outer - inner
    geom_time = dde.geometry.TimeDomain(t_begin, t_end)
    geom_full = dde.geometry.GeometryXTime(geom_space, geom_time)

    def boundary_edges(x, on_boundary):
        return on_boundary and (
            dde.utils.isclose(x[1], y_end) or dde.utils.isclose(x[1], y_begin)
        )

    def boundary_outflow(x, on_boundary):
        return on_boundary and dde.utils.isclose(x[0], x_end)

    def boundary_inflow(x, on_boundary):
        return on_boundary and dde.utils.isclose(x[0], x_begin)

    def boundary_inner(x, on_boundary):
        return on_boundary and inner.on_boundary(x)

    # BCs
    # No slip on the sphere
    bc_noslip = [
        dde.icbc.DirichletBC(geom_full, lambda x: 0, boundary_inner, component=0),
        dde.icbc.DirichletBC(geom_full, lambda x: 0, boundary_inner, component=1),
    ]
    # Inlet
    bc_in = [
        dde.icbc.DirichletBC(geom_full, lambda x: u_inf, boundary_inflow, component=0),
        dde.icbc.DirichletBC(geom_full, lambda x: 0, boundary_inflow, component=1),
        dde.icbc.OperatorBC(
            geom_full,
            lambda x, y, _: dde.grad.jacobian(y, x, i=2, j=0),
            boundary_outflow,
            # component=2,
        ),
    ]
    # Outlet
    bc_out = [
        dde.icbc.OperatorBC(
            geom_full,
            lambda x, y, _: dde.grad.jacobian(y, x, i=0, j=0),
            boundary_outflow,
            # component=0,
        ),
        dde.icbc.OperatorBC(
            geom_full,
            lambda x, y, _: dde.grad.jacobian(y, x, i=1, j=0),
            boundary_outflow,
            # component=1,
        ),
        dde.icbc.DirichletBC(geom_full, lambda x: 0, boundary_outflow, component=2),
    ]
    # Edges
    bc_edges = [
        dde.icbc.DirichletBC(geom_full, lambda x: u_inf, boundary_edges, component=0),
        dde.icbc.DirichletBC(geom_full, lambda x: 0, boundary_edges, component=1),
        dde.icbc.OperatorBC(
            geom_full,
            lambda x, y, _: dde.grad.jacobian(y, x, i=2, j=1),
            boundary_edges,
            # component=2,
        ),
    ]

    # Initial value
    ic = [
        dde.icbc.IC(
            geom_full, lambda x: u_inf, lambda _, on_initial: on_initial, component=0
        ),
        dde.icbc.IC(
            geom_full, lambda x: 0, lambda _, on_initial: on_initial, component=1
        ),
        dde.icbc.IC(
            geom_full, lambda x: 0, lambda _, on_initial: on_initial, component=2
        ),
    ]

    # Load data
    data = dde.data.TimePDE(
        geom_full,
        laminar_eq.equation,
        [
            # *bc_noslip,
            *bc_in,
            *bc_out,
            *bc_edges,
            *ic,
        ],
        num_domain=n_training_inside,
        num_boundary=n_training_bdy,
        num_initial=n_training_initial,
        solution=lambda x: [1, 1, 1],
        num_test=10,
    )

    neural_net = dde.nn.FNN(layers_sizes, activation, initializer)
    # neural_net.apply_output_transform(lambda x, y: abs(y))
    model = dde.Model(data, neural_net)
    model.compile("adam", **optimizer_kw)
    # model.train(iterations=n_iterations)
    # model.compile("L-BFGS")
    losshistory, train_state = model.train(iterations=n_iterations)

    return laminar_eq, model, losshistory, train_state

In [91]:
def get_predictions(
    laminar_eq: LaminarFlow2D, model: dde.Model, n_dim: int, n_time: int
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    x = np.linspace(x_begin, x_end, n_dim)
    y = np.linspace(y_begin, y_end, n_dim)
    t = np.linspace(t_begin, t_end, n_time)
    X, Y, T = np.meshgrid(x, y, t)
    grid = np.stack([X.flatten(), Y.flatten(), T.flatten()], axis=1)

    y_predict = model.predict(grid)
    y_predict = y_predict.reshape((n_dim, n_dim, n_time))
    return (x, y, t), y_predict

In [92]:
def mae(y_true: np.ndarray, y_predict: np.ndarray) -> float:
    return np.mean(np.abs(y_true - y_predict))


def plot_result(
    coords: np.ndarray, y_predict: np.ndarray, y_true: np.ndarray, n: int, kappa: float
) -> go.Figure:
    width = 600
    height = 600
    scale = 3

    fig = go.Figure(
        data=go.Heatmap(
            z=y_predict.T,
            x=coords[1],
            y=coords[0],
            colorscale="RdBu_r",
            zmin=-1,
            zmax=1,
            colorbar=dict(
                title=dict(text="Temperature difference", side="bottom"),
                orientation="h",
                x=0.5,
                y=-0.2,
                xanchor="center",
                yanchor="top",
            ),
        )
    )
    error = mae(y_true.flatten(), y_predict.flatten())
    fig.update_layout(
        autosize=False,
        width=width,
        height=height,
        title=dict(
            text=f"Case n={n}, diffusivity = {kappa:.1E}, MAE: {error:.2E}",
            y=0.91,
            x=0.5,
            xanchor="center",
            yanchor="bottom",
        ),
        xaxis=dict(title=dict(text="t")),
        yaxis=dict(title=dict(text="x")),
        font=dict(size=14),
        margin=dict(l=40, r=40, t=60, b=60),
    )
    fig.write_image(
        f"../figs/heat_equation_n={n}.png",
        width=width,
        height=height,
        scale=scale,
    )
    return fig

In [93]:
reynolds = 0.1
# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    # loss_weights=[0.001, 2, 0.5],
)
heat_eq, model, loss_history, train_state = solve_problem(reynolds=reynolds)
dde.saveplot(loss_history, train_state, issave=False, isplot=True)
# plot_result(train_state, name_case="underdamped")

Compiling model...
'compile' took 0.000096 s

Training model...

0         [4.39e+00, 1.01e+00, 2.97e-01, 1.14e+00, 9.21e-03, 1.93e-02, 5.11e-02, 3.93e-03, 2.29e-01, 2.23e+00, 4.99e-02, 9.52e-04, 2.24e+00, 7.74e-02, 9.93e-02][4.57e+00, 6.09e-01, 3.19e-01, 1.14e+00, 9.21e-03, 1.93e-02, 5.11e-02, 3.93e-03, 2.29e-01, 2.23e+00, 4.99e-02, 9.52e-04, 2.24e+00, 7.74e-02, 9.93e-02][1.84e+00, 1.78e+00, 1.79e+00]    
1000      [1.16e-03, 4.53e-04, 4.38e-05, 3.57e-05, 1.47e-05, 7.17e-05, 1.50e-04, 2.94e-05, 2.46e-05, 7.48e-05, 2.59e-05, 1.22e-04, 3.68e-05, 4.93e-06, 1.59e-04][1.58e-03, 1.84e-04, 5.55e-05, 3.57e-05, 1.47e-05, 7.17e-05, 1.50e-04, 2.94e-05, 2.46e-05, 7.48e-05, 2.59e-05, 1.22e-04, 3.68e-05, 4.93e-06, 1.59e-04][1.42e+00, 1.40e+00, 1.42e+00]    


KeyboardInterrupt: 

In [ ]:
coords, y_predict, y_true = get_predictions(heat_eq, model, 1000)
plot_result(coords, y_predict, y_true, n=1, kappa=0.1)

In [ ]:
n = 4
kappa = 0.025
# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    # loss_weights=[0.001, 2, 0.5],
)
heat_eq_2, model_2, loss_history_2, train_state_2 = solve_problem(kappa=kappa, n=n, kappa=kappa)

In [ ]:
dde.saveplot(loss_history_2, train_state_2, issave=False, isplot=True)
# plot_result(train_state, name_case="underdamped")

coords, y_predict, y_true = get_predictions(heat_eq_2, model_2, 1000)
plot_result(coords, y_predict, y_true, n=2)